## Idee per Progetti di Data Science sul Dataset Netflix

Ecco alcune idee di progetti di data science che puoi svolgere utilizzando il dataset Netflix con le colonne fornite: `show_id`, `type`, `title`, `director`, `cast`, `country`, `date_added`, `release_year`, `rating`, `duration`, `listed_in`, `description`.

### Analisi del Contenuto e delle Tendenze:

* **Analisi dei Generi** (`listed_in`, `type`):
    * Quali sono i generi più popolari (film vs serie TV)?
    * Come è cambiata la distribuzione dei generi nel tempo (`date_added`, `release_year`)?
    * Ci sono differenze nei generi popolari tra i diversi paesi (`country`)?
* **Analisi dei Registi e del Cast** (`director`, `cast`):
    * Quali sono i registi o gli attori più prolifici su Netflix?
    * C'è una correlazione tra la presenza di determinati attori/registi e il rating (`rating`)?
    * Analisi delle collaborazioni tra registi e attori.
* **Analisi Geografica** (`country`):
    * Quali paesi producono più contenuti?
    * Ci sono paesi specializzati in determinati generi?
    * Come varia la distribuzione dei rating tra i diversi paesi?
* **Analisi Temporale** (`date_added`, `release_year`):
    * Come è cresciuto il catalogo Netflix nel corso degli anni?
    * C'è una differenza significativa tra l'anno di produzione (`release_year`) e la data di aggiunta su Netflix (`date_added`)?
    * Analisi delle tendenze di rilascio (ci sono mesi o periodi con più aggiunte?).
* **Analisi dei Rating** (`rating`):
    * Qual è la distribuzione dei rating nel dataset?
    * C'è una correlazione tra il rating e altri fattori come genere, durata o paese?
    * Come è evoluta la distribuzione dei rating nel tempo?
* **Analisi della Durata** (`duration`):
    * Qual è la durata media dei film? Come varia per genere o paese?
    * Qual è la distribuzione del numero di stagioni per le serie TV?

### Sistemi di Raccomandazione di Base:

* **Raccomandazione basata sui Generi** (`listed_in`): Suggerire contenuti dello stesso genere che un utente ha apprezzato.
* **Raccomandazione basata sul Cast/Regista** (`cast`, `director`): Suggerire altri contenuti con gli stessi attori o registi.
* **Raccomandazione basata sul Paese** (`country`): Suggerire contenuti dallo stesso paese di origine.

### Analisi del Testo (`description`):

* **Word Cloud o Analisi delle Frequenze:** Quali sono le parole chiave più comuni nelle descrizioni?
* **Topic Modeling:** Identificare gli argomenti principali trattati nei contenuti basandosi sulle descrizioni.
* **Analisi del Sentiment** (più avanzato): Sebbene la descrizione sia oggettiva, potresti provare ad analizzare se alcune parole chiave implicano un certo tono o tema ricorrente.

Questi sono solo alcuni spunti. La scelta del progetto dipenderà dai tuoi interessi e dal livello di complessità che vuoi raggiungere.

In [36]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import math
import geopandas as gp
import geodatasets as gd

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
df = pd.read_csv('data/netflix_titles.csv')

url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world = gp.read_file(url)
#merging con il dataframe geopandas
countries_by_number_of_productions_df = pd.DataFrame(countries_by_number_of_productions.items(),columns=["SOVEREIGNT","productions_number"])
countries_by_number_of_productions_df.at[43,"SOVEREIGNT"] = "United States of America"
#gdf = countries_by_number_of_productions_df.merge(world,how="right",on="SOVEREIGNT")
gdf = world.merge(countries_by_number_of_productions_df,how="left",on="SOVEREIGNT")

NameError: name 'gp' is not defined

## Analisi del genere 

In [ ]:


# 📖 Gestione generi: separa i generi in liste
df['listed_in'] = df['listed_in'].fillna('')
df['genres'] = df['listed_in'].apply(lambda x: [i.strip() for i in x.split(',')])

# 📌 Esplodi le righe per ogni genere
df_exploded = df.explode('genres')

# 1️⃣ Generi più popolari (Film vs Serie)
genre_counts = df_exploded.groupby(['type', 'genres']).size().reset_index(name='count')
fig1 = px.bar(genre_counts, x='genres', y='count', color='type', barmode='group',
              title='🎥 Generi più popolari su Netflix (Film vs Serie)')
fig1.show()

# 2️⃣ Distribuzione dei generi nel tempo
genre_time = df_exploded.groupby(['release_year', 'genres']).size().reset_index(name='count')
genre_time = genre_time[genre_time['release_year'] >= 2000]  # opzionale: solo dal 2000 in poi

fig2 = px.line(genre_time, x='release_year', y='count', color='genres',
               title='📈 Distribuzione dei generi nel tempo (dal 2000)')
fig2.show()

# 3️⃣ Differenze nei generi per paese
df['listed_in'] = df['listed_in'].fillna('')
df['genres'] = df['listed_in'].apply(lambda x: [i.strip() for i in x.split(',')])
df_exploded = df.explode('genres')
df_exploded['main_country'] = df_exploded['country'].fillna('Unknown').apply(lambda x: x.split(',')[0].strip())

# Conta generi per paese
genre_country = df_exploded.groupby(['main_country', 'genres']).size().reset_index(name='count')

# Top 10 paesi con più titoli
top_countries = genre_country.groupby('main_country')['count'].sum().nlargest(10).index

# Filtra solo quei paesi
genre_country_top = genre_country[genre_country['main_country'].isin(top_countries)]

# Funzione per raggruppare top generi + 'Altro'
def top_n_and_other(df, n=5):
    top_genres = df.nlargest(n, 'count')
    other_count = df[~df['genres'].isin(top_genres['genres'])]['count'].sum()
    other_row = pd.DataFrame({'genres': ['Altro'], 'count': [other_count]})
    return pd.concat([top_genres[['genres', 'count']], other_row], ignore_index=True)

# Plotly figure
fig = go.Figure()

# Aggiungi una trace per ogni paese
for country in top_countries:
    country_data = genre_country_top[genre_country_top['main_country'] == country]
    summarized = top_n_and_other(country_data, n=5)
    fig.add_trace(go.Bar(x=summarized['genres'], y=summarized['count'], name=country, visible=False))

# Rendi visibile solo il primo
fig.data[0].visible = True

# Crea dropdown
buttons = []
for i, country in enumerate(top_countries):
    visible = [False] * len(fig.data)
    visible[i] = True
    buttons.append(dict(label=country, method="update", args=[{"visible": visible}, {"title": f"Top generi in {country}"}]))

# Layout
fig.update_layout(
    updatemenus=[dict(active=0, buttons=buttons)],
    title=f"Top generi in {top_countries[0]}",
    xaxis_title="Genere",
    yaxis_title="Numero di titoli",
    showlegend=False
)

fig.show()

Quanta percentuale di un genere e film o serie tv
Parole frequenti per periodo di tempo nltk(libreria pyton)
Trfdf


## Analisi dei Registi e del Cast

In [ ]:
# Pulizia e conteggio registi
director_counts = df['director'].dropna().value_counts().reset_index()
director_counts.columns = ['director', 'count']

# Prendi i primi 10
top_directors = director_counts.head(10)

# Grafico
fig = px.bar(top_directors, x='director', y='count',
             title='🎬 Top 10 Registi più prolifici su Netflix')
fig.show()

In [ ]:
# Pulizia e conteggio attori
df['cast'] = df['cast'].fillna('')
cast_series = df['cast'].str.split(', ').explode()

# Tieni solo stringhe non vuote e con almeno una lettera
cast_series = cast_series[cast_series.str.strip() != '']
cast_series = cast_series[cast_series.str.contains(r'[A-Za-z]', na=False)]

# Conta e prendi i primi 10
actor_counts = cast_series.value_counts().reset_index()
actor_counts.columns = ['actor', 'count']
top_actors = actor_counts.head(10)

# Grafico
fig = px.bar(top_actors, x='actor', y='count',
             title='🎭 Top 10 Attori più prolifici su Netflix (puliti)')
fig.show()

In [ ]:
# Funzione di mapping
def map_age_group(rating):
    if rating in ['TV-Y', 'TV-G', 'G']:
        return 'Bambini (0-7)'
    elif rating in ['TV-Y7', 'PG']:
        return 'Ragazzi (8-12)'
    elif rating in ['PG-13', 'TV-PG']:
        return 'Teen (13-17)'
    elif rating in ['R', 'NC-17', 'TV-14', 'TV-MA']:
        return 'Adulti (18+)'
    else:
        return 'Non classificato'

# Applica la mappatura
df['age_group'] = df['rating'].apply(map_age_group)

fig = px.histogram(df[df['actor_group'] != 'Altro'],
                   x='actor_group', color='age_group',
                   barmode='group',
                   title='🎭 Distribuzione Rating per Attore (Top 5, Fasce d’età)')
fig.show()


# Prendi i primi 5 registi più frequenti
top_5_directors = df['director'].value_counts().head(5).index

# Crea il grafico con i primi 5 registi
fig = px.histogram(df[df['director'].isin(top_5_directors)],
                   x='director', color='age_group',
                   barmode='group',
                   title='🎬 Distribuzione Rating per i Top 5 Registi (Fasce d’età)')
fig.show()


KeyError: 'actor_group'

In [ ]:
# Top 5 registi e attori
top5_directors = director_counts.head(5)['director'].tolist()

# Filtra titoli con entrambi
collab_df = df[df['director'].isin(top5_directors) & (df['cast'].notna())]

# Esplodi cast
collab_df['cast'] = collab_df['cast'].str.split(', ')
collab_df = collab_df.explode('cast')

# Filtra attori top 5
collab_df = collab_df[collab_df['cast'].isin(top5_actors)]

# Conta collaborazioni
collab_counts = collab_df.groupby(['director', 'cast']).size().reset_index(name='count')

# Heatmap
fig = px.imshow(collab_counts.pivot(index='director', columns='cast', values='count').fillna(0),
                text_auto=True, color_continuous_scale='Blues',
                title='🤝 Collaborazioni tra Registi e Attori')
fig.show()


C:\Users\loren\AppData\Local\Temp\ipykernel_26432\1527488383.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#creazione lista generi
movie_genres = []
for s in df[df["type"]=='Movie']["listed_in"]:
    movie_genres += s.split(', ')
movie_genres = list(set(list(movie_genres)))
movie_genres

NameError: name 'df' is not defined

In [ ]:
#ritorna durata in minuti del film
def getDurationInMinutes(duration_string):
    if isinstance(duration_string,str):
        return int(duration_string.split(' ')[0])
    return 0

#creazione dizionario contenente durata media in funzione del genere
average_movie_duration_per_genre = {genre:0 for genre in movie_genres}
for genre in movie_genres:
    durations = df[df["listed_in"].str.contains(genre)]["duration"].values.tolist()
    durations = [getDurationInMinutes(s) for s in durations if isinstance(s,str)]
    average_movie_duration_per_genre[genre] = np.mean(durations)
average_movie_duration_per_genre

{}

In [ ]:
#creazione dataframe dal dizionario
index = [i for i in range(0,len(average_movie_duration_per_genre))]
genres_average_duration_df = pd.DataFrame.from_dict(average_movie_duration_per_genre,orient='index')
genres_average_duration_df['i'] = index
genres_average_duration_df = genres_average_duration_df.reset_index().set_index('i')
genres_average_duration_df.columns = ['genre','average_duration']
m_i = genres_average_duration_df[genres_average_duration_df["genre"]=='Movies'].index
genres_average_duration_df.drop(m_i[0],inplace=True)
genres_average_duration_df

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [ ]:
#visualizza istogramma generi con rispettiva durata in minuti
px.histogram(genres_average_duration_df,y="genre",x="average_duration").update_yaxes(categoryorder='total ascending').update_layout(xaxis_title="Average duration (min)", yaxis_title="Movie genres")

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['index'] but received: average_duration

In [ ]:
#creazione lista direttori
directors = []
for s in df["director"]:
    if isinstance(s,str):
        directors += s.split(', ')
unique_directors = list(set(list(directors)))
#creazione dizionario contenente numero di produzioni in funzione del direttore
number_of_occurrences_per_director = {director:0 for director in unique_directors}
for director in unique_directors:
    number_of_occurrences_per_director[director] = directors.count(director)
number_of_occurrences_per_director = {k: v for k, v in sorted(number_of_occurrences_per_director.items(), key=lambda item: item[1],reverse=True)}
number_of_occurrences_per_director

NameError: name 'df' is not defined

In [ ]:
#creazione nuovo dataframe contenente i top 10 direttori
directors_df = pd.DataFrame(number_of_occurrences_per_director.items(), columns=["Name","Number of movies/TV series"])
top_10_directors_df = directors_df[:10]
top_10_directors_df

NameError: name 'number_of_occurrences_per_director' is not defined

In [ ]:
#visualizza tabella
fig = go.Figure(data=[go.Table(
    header=dict(values=list(top_10_directors_df.columns),
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=top_10_directors_df.transpose().values.tolist(),
               fill_color='lavender',
               align='center'))
])
fig.show()

NameError: name 'top_10_directors_df' is not defined

In [ ]:
movie_duration_by_release_year_df = df[df["type"]=='Movie'][["release_year","duration"]]
movie_duration_by_release_year_df["duration"] = [getDurationInMinutes(s) for s in df[df["type"]=='Movie']["duration"]]
#np.mean(movie_duration_by_release_year_df[movie_duration_by_release_year_df["release_year"]==2017]["duration"])
movie_duration_by_release_year_df

NameError: name 'df' is not defined

In [ ]:
#lista delle nazioni
list_of_countries = []
for country in df["country"]:
    if isinstance(country,str):
        list_of_countries += country.split(', ')
list_of_countries = list(set(list_of_countries))

NameError: name 'df' is not defined

In [ ]:
#creazione dizionario contenente numero di produzioni in funzione del Paese
countries_by_number_of_productions = {country:len(df[df["country"].str.contains(country).fillna(False)]) for country in list_of_countries}
len(df)

NameError: name 'df' is not defined

In [ ]:
#visualizzazione mappa coropletica
fig = px.choropleth(gdf,geojson=gdf.geometry,locations=gdf.index, color="productions_number",height=500,width=1500, range_color=(0,1000))
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(title_text = 'Number of movies/TV series produced by country',title_x = 0.5)
fig.show()

NameError: name 'gdf' is not defined